# Leaky Integrate-and-Fire Neuron

In this notebook we'll implement a *Leaky Integrate-and-Fire (LIF)* neuron with stochastic pre-synaptic input current(s) and extract and visualize its key statistical quantities.

This neuron model is defined by the *membrane equation* and *reset condition*:
\begin{align}
&\tau_m\,\frac{d}{dt}\,V(t) = E_{L} - V(t) + \frac{1}{g_L}\,I(t) &\text{if }\quad V(t) \leq V_{th}\\
\\
\\
&V(t) = V_{r} &\text{otherwise}\\
\\
\end{align}
where $V(t)$ is the membrane potential, $\tau_m$ is the membrane time constant, $E_{L}$ is the leak potential, $I(t)$ is the pre-synaptic input current, $V_{th}$ is the firing threshold and $V_{r}$ is the reset voltage.

We'll extract and visualize the mean, variance and histogram of the following quantities:

* Pre-synaptic input current(s) $I(t)$
* Membrane potential $V(t)$
* Output firing frequency $\lambda$

The problem will be split in several steps:

* ODE integration without spikes
* Stochastic pre-synaptic input
* Ensemble statistics
* Output spikes

## Simulation parameters

**Exercise**

Initialize the main simulation variables.
- $t_{max} = 0.1$ seconds
- $dt = 0.001$ seconds
- $\tau = 0.02$ seconds
- $E_L = -0.06$ volt
- $V_r = -0.07$ volt
- $V_{th} = -0.05$ volt
- $i_{mean} = 6.25\cdot 10^{-10}$ ampere
- $g_L = 25\cdot 10^{-9} $ siemens

## ODE integration without spikes

We want to solve LIF neuron's defining equation. Here, we'll do that numerically by replacing the time derivative $dV(t)/dt$ with the following approximation:


\begin{align*}
\tau_m\,\frac{V\left(t+\Delta t\right)-V\left(t\right)}{\Delta t} &= E_{L} - V(t) + \frac{1}{g_L}\,I(t)
\end{align*}

where $\Delta t$ is a small time step.
The value of the membrane potential $V\left(t+\Delta t\right)$ can then be expressed in terms of its previous value $V(t)$ by simple algebraic manipulation.

**EXERCISE**

Compute the values of $V(t)$ between $t=0$ and $t=0.01$ with $V(0)=E_L$ and pre-synaptic input given by:

$$
I(t)=I_{mean}\left(1+\sin\left(\frac{2 \pi}{0.01}\,t\right)\right)
$$

**Hints**
* Express $V\left(t+\Delta t\right)$ in terms of $V(t)$
* Initialize the membrane potential variable `V` with $E_L$
* Loop over the time variable `t` from `t=0` to `t=0.01` with time step $\Delta$`t`

* At each time step
    * Compute the current value of `I`
    * Update the value of `V`
    * Print `V`
* Use `np.pi` and `np.sin` for $\pi$ and $\sin(\,)$ respectively
* Expected output:
```
-0.05875
-0.0568277684346
-0.0545475593675
-0.0523813607538
-0.0507775611507
-0.0499886830932
-0.0499739805039
-0.0504141021241
-0.0508322176632
-0.0507753383454
-0.0507753383454
```

**EXERCISE**

Plot (instead of print) the values of $V(t)$ between $t=0$ and $t=t_{max}$ under the same conditions.

**Hint**
- Expected output
![lif](fig/lif_3.png)

## Stochastic pre-synaptic input

The pre-synaptic input received by neurons is random in nature. We'll replace the sinusoidal input from the previous exercise with a random input with similar statistical properties:

\begin{align}
I(t)=I_{mean}\left(1+0.1\sqrt{\frac{t_{max}}{\Delta t}}\,\xi(t)\right)\qquad\text{with }\xi(t)\sim U(-1,1)
\end{align}

where $\mathcal{U}$ denotes a uniform distribution. A random pre-synaptic input results in a random time course $V(t)$ for each execution.

**EXERCISE**

Plot the values of $V(t)$ between $t=0$ and $t=t_{max}$ with random input $I(t)=I_{mean}\left(1+0.1\sqrt{\frac{t_{max}}{\Delta t}}\,\xi(t)\right)\qquad\text{with }\xi(t)\sim U(-1,1)$.

The function `np.random.seed()` initializes the random number generator and `random.random()` generates a sample from a uniform distribution between `0` and `1`.

**Hints**
* Initialize the random number generator seed via ``np.random.seed``
* Use the function `np.random.random()` to generate the input at each timestep
* Repeat the execution several times to verify that $V(t)$ has a random time course
* Expected output for random seed 42
![lif](fig/lif_4.png)

## Ensemble statistics

Multiple runs of the previous exercise may give the impression of periodic regularity in the evolution of $V(t)$. We'll collect the sample mean over $N$ realizations of $V(t)$  with random input in order to test such hypothesis. The sample mean, sample variance and sample covariance at times $\left\{t_k, t_j\right\}\in[0,t_{max}]$ and for $N$ realizations of $V(t)$ are given by:

\begin{align}
\overline{\left\langle V(t_k)\right\rangle} &= \frac{1}{N}\sum_{n=1}^N V(t_k)_n & & \text{sample mean}\\
\overline{\left\langle (V(t_k)-\left\langle V(t_k)\right\rangle)^2\right\rangle} &= \frac{1}{N-1} \sum_{n=1}^N \left(V(t_k)_n-\left\langle V(t_k)\right\rangle\right)^2 & & \text{sample variance} \\
\overline{\left\langle \left(V(t_k)-\left\langle V(t_k)\right\rangle\right)\left(V(t_j)-\left\langle V(t_j)\right\rangle\right)\right\rangle}
&= \frac{1}{N-1} \sum_{n=1}^N \left(V(t_k)_n-\left\langle V(t_k)\right\rangle\right)\left(V(t_j)_n-\left\langle V(t_j)\right\rangle\right) & & \text{sample covariance}
\end{align}


**EXERCISE**

Visualize `N=100` realizations, as well as the mean and standard deviation of the voltage over realizations.

**Hints**
* At each timestep loop over `N` realizations of random input
* Plot each realization with keyword `'kx'` and parameter `alpha=0.05`
* Plot the sample mean at each timestep with keyword 'ro'
* Plot mean $\pm$ std with ```plt.fill_between``` in red with ```alpha=0.8```
* Expected output for random seed 42
![lif](fig/lif_5.png)

**EXERCISE**

Using $N=10000$ realizations, plot a histogram of $V(t)$ for $t=t_{max}/10$ and $t=t_{max}$. Use 25 bins.

**Hints**
* Plot the 1st histogram after running up to $t=t_{max}/10$
* Plot the 2nd histogram at $t=t_{max}$
* Add a legend to identify the time of each histogram
* Have a look at the parameters ```histtype``` and ```linewidth``` of ```plt.hist```
* Expected output
![lif](fig/lif_8.png)

## Output Spikes

The last step is to introduce the reset condition:
This neuron model is defined by the *membrane equation* and *reset condition* (which we have neglected up to here):

\begin{align}
V(t) &= V_{r} &\text{if }\quad V(t)\geq V_{th}
\end{align}

**EXERCISE**

Insert the reset condition and collect the spiking times of each realization in a dictionary variable `spikes`.

**Hints**
* Use $n=500$ realizations
* Reset $V(t)$ to $V_r$ if $V(t)\geq V_{th}$ and collect the spike time
* Plot each realization with keyword `'kx'` and parameter `alpha=0.05`
* Plot the sample mean with keyword `'ro'` in the same plot
* Create a new figure and plot the spike times with keyword `'k|'`
* In yet another figure plot the mean number of spikes at each time step
* Expected output
![lif](fig/lif_9a.png)
![lif](fig/lif_9b.png)

## OPTIONAL EXERCISES

* Explore the effects of
    - a refractory period
    - a smaller $\Delta t$
    - analytic integration of the LIF-ODEs

* Simulate a network of LIF neurons